Guides :

- https://openpyxl.readthedocs.io/en/stable/
- https://automatetheboringstuff.com/chapter12/

# Import des librairies

In [ ]:
from pathlib import Path

import openpyxl
import pandas as pd

# Variables globales

In [ ]:
data = Path("../data")  # Chemin vers le dossier data


# Exploration du fichier excel des modules

## Variables globales

In [ ]:
colonnes = [
    'Identifiant',
    'Abréviation',
    'Code PPN',
    'Nom complet',
    'Promotion',
    'Période',
    'Responsable'
]

In [ ]:
module_wb = openpyxl.load_workbook(
    data / "liste_modules.xlsx"
)  # Objet python représentant le fichier excel
print(
    module_wb.sheetnames
)  # On récupère le nom des feuilles du fichier excel, ici il n'y en a qu'une seule
module_sheet = module_wb[
    module_wb.sheetnames[0]
]  # Objet python représentant la première feuille du fichier excel


# Obtention des indices de début pour chaque filière

In [ ]:
currentPathway = module_sheet[f'C{1}'].value

pathways = {currentPathway:[1]}

for i in range(1, module_sheet.max_row+1):
    if module_sheet[f'C{i}'].value != currentPathway:
        pathways[currentPathway].append(i-1)
        currentPathway = module_sheet[f'C{i}'].value
        pathways[currentPathway] = [i]

pathways[currentPathway].append(module_sheet.max_row)

print(pathways)

# Obtention des modules d'informatique

In [ ]:
selectedPathway = 'INFO'

firstRowCS = pathways[selectedPathway][0]
lastColumnCoordinates = f'{openpyxl.utils.cell.get_column_letter(module_sheet.max_column)}{firstRowCS}'
firstLine = module_sheet[f'A{firstRowCS}':lastColumnCoordinates]
for rowOfCellObjects in firstLine:
    for cellObj in rowOfCellObjects:
        print(cellObj.coordinate, cellObj.value)

## Mise en place de la structure de donnée interne

In [ ]:
demonstration_df = pd.DataFrame(columns=colonnes)
demonstration_df.head()

### Informations directement disponibles

In [ ]:
# Depuis les données brutes on peut remplir les 2 premières colonnes
rawData = {
    'Identifiant': firstLine[0][0].value,
    'Abréviation' : firstLine[0][1].value
}

currentEntry = pd.DataFrame(rawData, index=[0]) # On créé un dataframe contenant les données bruts

demonstration_df = pd.concat([demonstration_df, currentEntry]).reset_index(drop=True) # On l'ajoute au dataframe principal

demonstration_df

In [ ]:
# Dans notre cas des informations sont redondantes

demonstration_df['Code PPN'] = demonstration_df['Identifiant']
demonstration_df['Nom complet'] = demonstration_df['Abréviation']

demonstration_df

### Informations à déduire

In [ ]:
description = firstLine[0][11].value

print(description)

descriptionSegmented = description.replace('/', ',').split(',')

print(descriptionSegmented)

# Removing
indexToRemove = []

for i in range(len(descriptionSegmented)):
    processedString = descriptionSegmented[i].strip()

    if processedString == '':
        indexToRemove.append(i)
    else:
        descriptionSegmented[i] = processedString

for index in reversed(indexToRemove):
    descriptionSegmented.pop(index)

print(descriptionSegmented)

In [ ]:
semesterSegment = descriptionSegmented[1]
print(semesterSegment.strip('Semestre').strip())
semesterNumber = semesterSegment.strip('Semestre').strip()[1]

currentEntry['Période'] = f'S{semesterNumber}'

year = int(semesterNumber)%2 # 2 semestres par année

yearGroup = f'{selectedPathway}{year}'
print(yearGroup)

currentEntry['Promotion'] = yearGroup

demonstration_df = pd.concat([demonstration_df, currentEntry]).reset_index(drop=True) # On l'ajoute au dataframe principal

demonstration_df

Pour le responsable on s'attend à son identifiant, il faut implémenter une fonction permettant de récupérer son identifiant; on ne peut pas juste prendre ses initiales (cas où 2 individus ont les mêmes initiales)

In [ ]:
identity = firstLine[0][4].value

#TODO: recherche d'identifiant

## Regroupement des procédures d'extraction d'informations

In [ ]:
def extractModuleInformation(moduleExcelLine: tuple):
    moduleLine = moduleExcelLine[0]
    moduleDataFrame = pd.DataFrame(columns=colonnes)
    moduleDataFrame.head

    rawData = {
        'Identifiant': moduleLine[0].value,
        'Abréviation' : moduleLine[1].value
    }

    rawDataFrame = pd.DataFrame(rawData, index=[0]) # On créé un dataframe contenant les données bruts

    moduleDataFrame = pd.concat([moduleDataFrame, rawDataFrame]).reset_index(drop=True) # On l'ajoute au dataframe principal

    moduleDataFrame["Code PPN"] = moduleDataFrame["Identifiant"]
    moduleDataFrame["Nom complet"] = moduleDataFrame["Abréviation"]

    description = moduleLine[11].value

    #print(description)

    descriptionSegmented = description.replace("/", ",").split(",")

    #print(descriptionSegmented)

    # Removing
    indexToRemove = []

    for i in range(len(descriptionSegmented)):
        processedString = descriptionSegmented[i].strip()

        if processedString == "":
            indexToRemove.append(i)
        else:
            descriptionSegmented[i] = processedString

    for index in reversed(indexToRemove):
        descriptionSegmented.pop(index)

    semesterSegment = descriptionSegmented[1]
    #print(semesterSegment.strip("Semestre").strip())
    semesterNumber = semesterSegment.strip("Semestre").strip()[1]

    moduleDataFrame["Période"] = f"S{semesterNumber}"

    year = int(semesterNumber) % 2  # 2 semestres par année

    yearGroup = f"{selectedPathway}{year}"
    #print(yearGroup)

    moduleDataFrame["Promotion"] = yearGroup

    # TODO: extraire les informations pour le reponsable de module

    return moduleDataFrame


In [ ]:
extractModuleInformation(firstLine).head()